# Merging Instacart Data 

# Instacart — Part 1: Merge `orders_wrangled` with `orders_products_prior`

**Goal:** Create `orders_products_combined.pkl` by merging orders with prior order line-items.

**Why:** Analyze all Instacart orders and their products in one place.

**Inputs**
- `02 Data/Prepared Data/orders_wrangled.csv`
- `02 Data/Original Data/orders_products_prior.csv`  *(unzipped per exercise)*

**Output**
- `02 Data/Prepared Data/orders_products_combined.pkl`

**Checklist**
1. Load data
2. Sanity checks (`head`, `shape`)
3. Merge on `order_id` with `indicator=True`
4. (One-time) coverage check with `how='outer'`
5. Export as pickle


In [29]:
import os, gc
import pandas as pd


path = r"C:\Users\Neway\OneDrive\Desktop\instacart Basket Analysis"  

assert os.path.isdir(path), f"Path does not exist: {path}"

prepared_dir = os.path.join(path, '02 Data', 'Prepared Data')
original_dir = os.path.join(path, '02 Data', 'Original Data')
os.makedirs(prepared_dir, exist_ok=True)


## Load data
We use the prepared (cleaned) orders and the new prior line-items.


In [31]:
orders_file = os.path.join(prepared_dir, 'orders_wrangled.csv')
prior_file  = os.path.join(original_dir, 'orders_products_prior.csv')

df_ords = pd.read_csv(orders_file)
df_ords_prior = pd.read_csv(prior_file)

df_ords.head(3), df_ords.shape, df_ords_prior.head(3), df_ords_prior.shape


(   Unnamed: 0  order_id  user_id  order_number  orders_day_of_week  \
 0           0   2539329        1             1                   2   
 1           1   2398795        1             2                   3   
 2           2    473747        1             3                   3   
 
    order_hour_of_day  days_since_prior_order  
 0                  8                     NaN  
 1                  7                    15.0  
 2                 12                    21.0  ,
 (3421083, 7),
    order_id  product_id  add_to_cart_order  reordered
 0         2       33120                  1          1
 1         2       28985                  2          1
 2         2        9327                  3          0,
 (32434489, 4))

## Merge (inner join on `order_id`)
We expect all rows in the result to exist in both inputs. We add `_merge` to inspect behavior.


In [33]:
df_merged_large = df_ords.merge(
    df_ords_prior,
    on='order_id',
    how='inner',
    indicator=True
)
df_merged_large.head(3), df_merged_large.shape, df_merged_large['_merge'].value_counts()


(   Unnamed: 0  order_id  user_id  order_number  orders_day_of_week  \
 0           0   2539329        1             1                   2   
 1           0   2539329        1             1                   2   
 2           0   2539329        1             1                   2   
 
    order_hour_of_day  days_since_prior_order  product_id  add_to_cart_order  \
 0                  8                     NaN         196                  1   
 1                  8                     NaN       14084                  2   
 2                  8                     NaN       12427                  3   
 
    reordered _merge  
 0          0   both  
 1          0   both  
 2          0   both  ,
 (32434489, 11),
 _merge
 both          32434489
 left_only            0
 right_only           0
 Name: count, dtype: int64)

## 5 Coverage check (outer join test)
Inner joins always show `_merge == both`. Run an **outer** test once to confirm true coverage.


In [34]:
coverage = pd.merge(
    df_ords[['order_id']].drop_duplicates(),
    df_ords_prior[['order_id']].drop_duplicates(),
    on='order_id',
    how='outer',
    indicator=True
)['_merge'].value_counts()

coverage


_merge
both          3214874
left_only      206209
right_only          0
Name: count, dtype: int64

# Inspect non-mathcing ids

In [36]:
outer_test = pd.merge(
    df_ords[['order_id']].drop_duplicates().assign(source='orders'),
    df_ords_prior[['order_id']].drop_duplicates().assign(source='prior'),
    on='order_id',
    how='outer',
    indicator=True
)
outer_test.query("_merge != 'both'").head()


,order_id,source_x,source_y,_merge
0,1,orders,NaN,left_only
16,17,orders,NaN,left_only
33,34,orders,NaN,left_only
35,36,orders,NaN,left_only
37,38,orders,NaN,left_only


# 6 Export to pickle

## Export
We export as `.pkl` for speed and lossless round-trip.


In [37]:
out_pkl = os.path.join(prepared_dir, 'orders_products_combined.pkl')
df_merged_large.to_pickle(out_pkl)

# (Optional portability)
# df_merged_large.to_csv(os.path.join(prepared_dir, 'orders_products_combined.csv'), index=False)

del df_ords, df_ords_prior
gc.collect()
out_pkl


'C:\\Users\\Neway\\OneDrive\\Desktop\\instacart Basket Analysis\\02 Data\\Prepared Data\\orders_products_combined.pkl'